In [6]:
!pip install -r requirements.txt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [3]:

import os
from dotenv import load_dotenv
load_dotenv() ## load enivronment variables

True

In [5]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [7]:
## Data ingestion -- from website scrap the data

from langchain_community.document_loaders import WebBaseLoader

In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")

In [9]:
loader

In [10]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this content, it might be helpful to read the following:Data Retention Conceptual DocsUsage Limiting Conceptual DocsnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of 

In [11]:
# divide it into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [13]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spe

In [16]:
##convert to vectors
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()


In [17]:
## FIASS DB to store vectors
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(documents,embeddings)

In [18]:
db

In [19]:
query="LangSmith has two usage limits: total traces and extended"
result=db.similarity_search(query)
result[0].page_content

'That\'s a cost reduction of nearly 75% per day!Optimization 2: limit usage‚ÄãIn the previous section, we managed data retention settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future overspend.LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along\nwith a cost estimate:Lets start by setting limits on our production usage, since that is where the majority of spend comes from.Setting a good total traces limit‚ÄãPicking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You should'

In [21]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [23]:
## Retrieval chain, Documents chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template(
    """
    Answer the following question based only on provided context:
    <context>
    {context}
    </context>

    """
)
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the following question based only on provided context:\n    <context>\n    {context}\n    </context>\n    \n    '))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7d1a2a773fa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d1a2a771990>, root_client=<openai.OpenAI object at 0x7d1a2a7707f0>, root_async_client=<openai.AsyncOpenAI object at 0x7d1a2a773e20>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [25]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended.These correspond to the two metrics we have been tracking on our usage graph")]

})

'    LangSmith has two usage limits: total traces and extended. These correspond to the two metrics being tracked on the usage graph.'

In [26]:
### Input-->Retriever-->db

from langchain.chains import create_retrieval_chain
retriever=db.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d1a38aa5900>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the following question based only on provided context:\n    <context>\n    {context}\n    </context>\n    \n    '))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7d1a2a773fa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at

In [30]:
## get the response from the llm
response = retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'What are the two types of usage limits that LangSmith offers?\n\nLangSmith offers two usage limits: total traces and extended retention traces.'